In [12]:
import pandas as pd
import polars as pl
from data import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pyarrow 
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import os
import numpy as np
from scipy import stats
from typing import cast
from utils import *
import wrds

In [2]:
db = wrds.Connection(
    wrds_username='jayati'
)


Loading library list...
Done


In [3]:
crsp_monthly_query = """
SELECT msf.permno, msf.date, ssih.siccd
FROM crsp.msf AS msf
LEFT JOIN crsp.stksecurityinfohist AS ssih
ON msf.permno = ssih.permno
AND ssih.secinfostartdt <= msf.date AND msf.date <= ssih.secinfoenddt
WHERE msf.date BETWEEN '1986-01-01' AND '2024-05-31'
"""
crsp_monthly_data = db.raw_sql(crsp_monthly_query)

crsp_monthly_df = pl.from_pandas(crsp_monthly_data)

crsp_monthly_df.select(['permno', 'date', 'siccd']).write_csv('data/crsp_monthly_with_industries.csv')


In [4]:
crsp_monthly_df.height

3487273

In [5]:
complete_data = pl.read_csv('data/COMPLETE_DATASET.csv')
complete_data = complete_data.with_columns([
    pl.col('date').str.strptime(pl.Date, "%Y-%m-%d")
])

crsp_monthly_df = pl.read_csv('data/crsp_monthly_with_industries.csv')
crsp_monthly_df = crsp_monthly_df.with_columns([
    pl.col('date').str.strptime(pl.Date, "%Y-%m-%d")
])

merged_data = complete_data.join(
    crsp_monthly_df.select(['permno', 'date', 'siccd']),
    on=['permno', 'date'],
    how='left'
)


In [6]:
print(merged_data.head())


shape: (5, 217)
┌────────┬────────┬──────┬──────┬───┬───────────┬────────────┬───────────┬────────┐
│ permno ┆ yyyymm ┆ AM   ┆ AOP  ┆ … ┆ Size      ┆ date       ┆ ret       ┆ siccd  │
│ ---    ┆ ---    ┆ ---  ┆ ---  ┆   ┆ ---       ┆ ---        ┆ ---       ┆ ---    │
│ i64    ┆ i64    ┆ f64  ┆ str  ┆   ┆ f64       ┆ date       ┆ f64       ┆ f64    │
╞════════╪════════╪══════╪══════╪═══╪═══════════╪════════════╪═══════════╪════════╡
│ 10000  ┆ 198601 ┆ null ┆ null ┆ … ┆ -9.686575 ┆ 1986-01-31 ┆ null      ┆ 3990.0 │
│ 10000  ┆ 198602 ┆ null ┆ null ┆ … ┆ -9.389323 ┆ 1986-02-28 ┆ -0.257143 ┆ 3990.0 │
│ 10000  ┆ 198603 ┆ null ┆ null ┆ … ┆ -9.700759 ┆ 1986-03-31 ┆ 0.365385  ┆ 3990.0 │
│ 10000  ┆ 198604 ┆ null ┆ null ┆ … ┆ -9.627207 ┆ 1986-04-30 ┆ -0.098592 ┆ 3990.0 │
│ 10000  ┆ 198605 ┆ null ┆ null ┆ … ┆ -9.375336 ┆ 1986-05-30 ┆ -0.222656 ┆ 3990.0 │
└────────┴────────┴──────┴──────┴───┴───────────┴────────────┴───────────┴────────┘


In [7]:
sic_groups = {
    'Agriculture, Forestry, and Fishing': range(100, 1000),
    'Mining': range(1000, 1500),
    'Construction': range(1500, 1800),
    'Manufacturing': range(2000, 4000),
    'Transportation, Communications, Electric, Gas, and Sanitary Services': range(4000, 5000),
    'Wholesale Trade': range(5000, 5200),
    'Retail Trade': range(5200, 6000),
    'Finance, Insurance, and Real Estate': range(6000, 6800),
    'Services': range(7000, 9000),
    'Public Administration': range(9000, 10000)
}

In [8]:
def map_sic_to_industry(sic):
    for industry, sic_range in sic_groups.items():
        if sic in sic_range:
            return industry
    return 'Unknown'

# Step 5: Apply the mapping function to create an 'industry' column
merged_data = merged_data.with_columns([
    pl.col('siccd').apply(map_sic_to_industry).alias('industry')
])

# Step 6: Count the number of stocks in each industry
industry_counts = merged_data.groupby('industry').agg(pl.count('permno').alias('count'))
print(industry_counts)

/var/folders/b0/4ztxyrhx5zb29cnt86sf6m580000gn/T/ipykernel_1087/3215136193.py:9: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col('siccd').apply(map_sic_to_industry).alias('industry')
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


shape: (12, 2)
┌─────────────────────────────────┬─────────┐
│ industry                        ┆ count   │
│ ---                             ┆ ---     │
│ str                             ┆ u32     │
╞═════════════════════════════════╪═════════╡
│ Unknown                         ┆ 8847    │
│ Mining                          ┆ 169538  │
│ Construction                    ┆ 29898   │
│ Services                        ┆ 429601  │
│ Manufacturing                   ┆ 985827  │
│ …                               ┆ …       │
│ null                            ┆ 41902   │
│ Transportation, Communications… ┆ 234129  │
│ Retail Trade                    ┆ 148328  │
│ Finance, Insurance, and Real E… ┆ 1163369 │
│ Wholesale Trade                 ┆ 99408   │
└─────────────────────────────────┴─────────┘


/var/folders/b0/4ztxyrhx5zb29cnt86sf6m580000gn/T/ipykernel_1087/3215136193.py:13: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  industry_counts = merged_data.groupby('industry').agg(pl.count('permno').alias('count'))


In [9]:
merged_data.write_csv('data/UPDATED_COMPLETE_DATA_WITH_INDUSTRY.csv')



In [10]:
merged_data.head()

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,date,ret,siccd,industry
i64,i64,f64,str,f64,f64,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,str,str,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,f64,f64,f64,f64,f64,str,str,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,date,f64,f64,str
10000,198601,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,-1.475907,-9.686575,1986-01-31,null,3990.0,"""Manufacturing"""
10000,198602,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,25.7143,-1.178655,-9.389323,1986-02-28,-0.257143,3990.0,"""Manufacturing"""
10000,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-36.5385,-1.490091,-9.700759,1986-03-31,0.365385,3990.0,"""Manufacturing"""
10000,198604,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null,9.8592,-1.386294,-9.627207,1986-04-30,-0.098592,3990.0,"""Manufacturing"""
10000,198605,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,0.008818,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6496e-8,null,22.2656,-1.134423,-9.375336,1986-05-30,-0.222656,3990.0,"""Manufacturing"""


In [14]:
industries = merged_data.select(pl.col('industry')).unique().to_series().to_list()

os.makedirs('data/industry_datasets', exist_ok=True)

for industry in industries:
    if industry is not None:
        industry_data = merged_data.filter(pl.col('industry') == industry)
        industry_filename = f"data/industry_datasets/{industry.replace(' ', '_')}_data.csv"
        industry_data.write_csv(industry_filename)
        print(f"Data for {industry} saved to {industry_filename}")


Data for Mining saved to data/industry_datasets/Mining_data.csv
Data for Wholesale Trade saved to data/industry_datasets/Wholesale_Trade_data.csv
Data for Manufacturing saved to data/industry_datasets/Manufacturing_data.csv
Data for Retail Trade saved to data/industry_datasets/Retail_Trade_data.csv
Data for Construction saved to data/industry_datasets/Construction_data.csv
Data for Transportation, Communications, Electric, Gas, and Sanitary Services saved to data/industry_datasets/Transportation,_Communications,_Electric,_Gas,_and_Sanitary_Services_data.csv
Data for Unknown saved to data/industry_datasets/Unknown_data.csv
Data for Agriculture, Forestry, and Fishing saved to data/industry_datasets/Agriculture,_Forestry,_and_Fishing_data.csv
Data for Public Administration saved to data/industry_datasets/Public_Administration_data.csv
Data for Services saved to data/industry_datasets/Services_data.csv
Data for Finance, Insurance, and Real Estate saved to data/industry_datasets/Finance,_In